In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
import itertools

In [ ]:
# Generate bufferbloat experiment.
experiment   = 'results/bufferbloat'
buffer_sizes = [5, 10, 15, 20, 25]
trial_nums   = list(range(10))
protocols    = ['vivace', 'allegro', 'cubic', 'reno']

PACKET_DATA_LENGTH = 1400

entries = []
for buf, trial, protocol in itertools.product(buffer_sizes, trial_nums, protocols):
    client_file = '{}/{}-{}-client-{}.json'.format(experiment, buf, protocol, trial)
    server_file = '{}/{}-{}-server-{}.json'.format(experiment, buf, protocol, trial)
    with open(client_file, 'r') as f:
        try:
            client_parsed = json.loads(f.read())
        except json.JSONDecodeError:
            print('Skipping file {}'.format(client_file))
    with open(server_file, 'r') as f:
        try:
            server_parsed = json.loads(f.read())
        except json.JSONDecodeError:
            print('Skipping file {}'.format(server_file))
            
    bytes_sent = client_parsed['total_packets'] * PACKET_DATA_LENGTH
    bytes_recv = server_parsed['total_packets'] * PACKET_DATA_LENGTH

    entries.append({
        'Protocol': protocol,
        'Trial': trial,
        'Buffer Size': buf,
        'Throughput': bytes_recv * 8 / server_parsed['time_since_start'],
        'Loss Rate': 1 - bytes_recv / bytes_sent,
    })
    
raw_df = pd.DataFrame(entries)

entries = []
for buf, protocol in itertools.product(buffer_sizes, protocols):
    trials = raw_df[(raw_df['Protocol'] == protocol) & (raw_df['Buffer Size'] == buf)]
    entries.append({
        'Protocol': protocol,
        'Buffer Size': buf,
        'AvgThroughput': trials['Throughput'].median(),
        'MinThroughput': trials['Throughput'].min(),
        'MaxThroughput': trials['Throughput'].max(),
        'AvgLoss': trials['Loss Rate'].median(),
        'MinLoss': trials['Loss Rate'].min(),
        'MaxLoss': trials['Loss Rate'].max(),
    })
    
parsed_df = pd.DataFrame(entries)

colors = {
    'vivace': 'blue',
    'allegro': 'red',
    'cubic': 'green',
    'reno': 'pink',
}

params = {
  'axes.labelsize': 18,
  'font.size': 18,
  'legend.fontsize': 18,
  'xtick.labelsize': 12,
  'ytick.labelsize': 12,
  'text.usetex': True,
  'figure.figsize': [6, 4],
  'text.latex.preamble': r'\usepackage{amsmath} \usepackage{amssymb}',
   }
plt.rcParams.update(params)
fig = plt.figure(figsize=(10,10))

plt.axes(frameon=0) # turn off frames
plt.grid(axis='y', color='0.9', linestyle='-', linewidth=1)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))

for protocol in protocols:
    rows = parsed_df[(parsed_df['Protocol'] == protocol)]
    plt.errorbar(rows['Buffer Size'], rows['AvgThroughput'], color=colors[protocol], label=protocol)
    
plt.title('Median Throughput vs Buffer Size for Bufferbloat')
plt.xlabel('Buffer Size (in packets)')
plt.ylabel('Median Throughput across 10 runs (in Mbps)')
plt.legend()
plt.savefig('bufferbloat_throughput.pdf')

fig = plt.figure(figsize=(10,10))

plt.axes(frameon=0) # turn off frames
plt.grid(axis='y', color='0.9', linestyle='-', linewidth=1)

for protocol in protocols:
    rows = parsed_df[(parsed_df['Protocol'] == protocol)]
    plt.errorbar(rows['Buffer Size'], rows['AvgLoss'], color=colors[protocol], label=protocol)
    
plt.title('Median Loss Rate vs Buffer Size for Bufferbloat')
plt.xlabel('Buffer Size (in packets)')
plt.ylabel('Median Loss Rate across 10 runs (Lower is better)')
plt.legend()
plt.savefig('bufferbloat_lossrate.pdf')

In [ ]:
# Generate bufferbloat experiment.
experiment   = 'results/bufferbloat'
buffer_sizes = [5, 10, 15, 20, 25]
trial_nums   = list(range(10))
protocols    = ['vivace', 'allegro', 'cubic', 'reno']

PACKET_DATA_LENGTH = 1400

entries = []
for buf, trial, protocol in itertools.product(buffer_sizes, trial_nums, protocols):
    client_file = '{}/{}-{}-client-{}.json'.format(experiment, buf, protocol, trial)
    server_file = '{}/{}-{}-server-{}.json'.format(experiment, buf, protocol, trial)
    with open(client_file, 'r') as f:
        try:
            client_parsed = json.loads(f.read())
        except json.JSONDecodeError:
            print('Skipping file {}'.format(client_file))
    with open(server_file, 'r') as f:
        try:
            server_parsed = json.loads(f.read())
        except json.JSONDecodeError:
            print('Skipping file {}'.format(server_file))
            
    bytes_sent = client_parsed['total_packets'] * PACKET_DATA_LENGTH
    bytes_recv = server_parsed['total_packets'] * PACKET_DATA_LENGTH

    entries.append({
        'Protocol': protocol,
        'Trial': trial,
        'Buffer Size': buf,
        'Throughput': bytes_recv * 8 / server_parsed['time_since_start'],
        'Loss Rate': 1 - bytes_recv / bytes_sent,
    })
    
parsed_df = pd.DataFrame(entries)

colors = {
    'vivace': 'blue',
    'allegro': 'red',
    'cubic': 'green',
    'reno': 'pink',
}

params = {
  'axes.labelsize': 18,
  'font.size': 18,
  'legend.fontsize': 18,
  'xtick.labelsize': 12,
  'ytick.labelsize': 12,
  'text.usetex': True,
  'figure.figsize': [6, 4],
  'text.latex.preamble': r'\usepackage{amsmath} \usepackage{amssymb}',
   }
plt.rcParams.update(params)
fig = plt.figure(figsize=(10,10))

plt.axes(frameon=0) # turn off frames
plt.grid(axis='y', color='0.9', linestyle='-', linewidth=1)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))

for protocol in protocols:
    rows = parsed_df[(parsed_df['Protocol'] == protocol) & (parsed_df['Buffer Size'] == 25)]
    plt.scatter(rows['Protocol'], rows['Throughput'], color=colors[protocol], label=protocol)
    
plt.title('Throughput of All Trials with Buffer Size 125')
plt.xlabel('Protocol')
plt.ylabel('Throughput (in Mbps)')
plt.savefig('bufferbloat_throughput_125variance.pdf')

fig = plt.figure(figsize=(10,10))

plt.axes(frameon=0) # turn off frames
plt.grid(axis='y', color='0.9', linestyle='-', linewidth=1)

for protocol in protocols:
    rows = parsed_df[(parsed_df['Protocol'] == protocol) & (parsed_df['Buffer Size'] == 25)]
    plt.scatter(rows['Protocol'], rows['Loss Rate'], color=colors[protocol], label=protocol)
    
plt.title('Loss Rate of All Trials with Buffer Size 125')
plt.xlabel('Protocol')
plt.ylabel('Loss Rate (Lower is better)')
plt.savefig('bufferbloat_lossrate_125variance.pdf')